## "Elbow Detector" Method
High-pass filter the two signals to:
- Remove any DC offset between the signals
- Isolate **slope changes** (or "elbows") in the two signals and compare those directly, as constant-slope comparisons may make the horizontal vs. vertical offsets estimation ambiguous in real-time.

## Kalman Filter Method
References:
- https://beteje.github.io/assets/pdf/2019_APSIPA.pdf
- https://www.sciencedirect.com/science/article/pii/S1474667016396033